# Orders API with Planet SDK 

Getting started with the Orders API using the Planet SDK.
This tutorial uses [Planet](https://www.planet.com)'s Orders API with the official [Python client](https://github.com/planetlabs/planet-client-python).

## Requirements

This tutorial assumes familiarity with the [Python](https://python.org) programming language throughout. Python modules used in this tutorial are:
* [IPython](https://ipython.org/) and [Jupyter](https://jupyter.org/)
* [planet-python-client](https://github.com/planetlabs/planet-client-python)

You should also have an account on the Planet Platform. You will login to create an access token that authenticates your API key from the [account page](https://www.planet.com/account/).

## Useful links 
* [Planet SDK and Developer Resources](https://docs.planet.com/develop/sdks/#planet-sdk-for-python-and-cli/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Orders API Documentation](https://docs.planet.com/develop/apis/orders/)

___

# Set up

In order to interact with the Planet API using the client, we need to import the necessary packages:

In [ ]:
import planet
import os
from planet import Auth, Planet, order_request, reporting, Session, OrdersClient

## SDK Authentication

We next need to create an OAuth2 access token using our Planet login for python client access. 
If you are not currently logged in to your Planet account, you will be prompted via a login link to do so. 
For more information, please visit https://docs.planet.com/develop/authentication

In [ ]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=True)
if not auth.is_initialized():
    auth.user_login(allow_open_browser=False, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

## Building an Order request

#### Manual JSON Order requests

You can build an Order request manually by using a blob of JSON -- you may have used this with Python Client V1, or with `requests` directly to the API:

In [ ]:
# No need to write this all out with the SDK
request = {
  "name": "test_order_sdk_method_1",
  "products": [
    {
     "item_ids":[  
        "20200922_183724_23_106a",
        "20200922_183722_17_106a"
         ],
      "item_type": "PSScene",
      "product_bundle": "analytic_udm2"
    }
  ],
   "tools": [
    {
      "reproject": {
        "projection": "EPSG:4326",
        "kernel": "cubic"
      }
    }
  ]
}

### Orders with Planet SDK

In SDK V3, you can build an Order request object using the new `build_request` functionality. You can specify all Order details including products, bundles/fallback bundles, cloud delivery configuration, tools & toolchain operations, etc. [Read more in the docs here](https://docs.planet.com/develop/apis/orders/).

#### Build the Order Request

Here we will build an order request. We will define for this example:
- Products, with 2 `item_ids`,  with the `product_bundle` of 'analytic_udm2', and the `item_type` of 'PSScene'
- Optional `tools`, which in this case is the `reproject_tool` 
    - `order_request.clip_tool` could also be used here and added to the `tools` list.
    - This would require a GeoJson aoi.
<br>

- We will not be providing a delivery location with `order_request.delivery`, `order_request.google_cloud_storage`, etc. 
- A download link with the `_links` key in the request output is provided instead for this example.

In [ ]:
# Define request details to properly build your request

item_ids = ["20200922_183724_23_106a", "20200922_183722_17_106a"]

products = [
    order_request.product(item_ids, 'analytic_udm2', 'PSScene')
]

tools = [
    order_request.reproject_tool(projection='EPSG:4326', kernel='cubic')
]

request = order_request.build_request(
    'test_order_sdk_method_2', products=products, tools=tools)


In [ ]:
# View your built order_request:
request

### Create the Order

The next step after building an Order is to send it to the Orders API to create it using 'create_order'.

To do this, we'll create a `Session` to manage our communcation with Planet in general -- this will make use of that `auth` object we created earlier. Within the context of this Session, we'll create an Orders API-specific `client` to handle interactions with the Orders API.

After the order is created, we can view the returned dictionary of created_order, which will contain the new `order_id`.

In [ ]:
# an async Orders client to request order creation
async def main():
  async with Session(auth=auth) as sess:
    cl = OrdersClient(sess)
    order = await cl.create_order(request)

    return order
# async magic to remember: "async def" to create a coroutine, then "await" to make it run
created_order = await main()

Congratulations! You have created an order. You can view it below:

In [ ]:
created_order

## Manually get the Order after it is created

You may have noticed that the `state` key in order you created is 'queued' when the order is first returned. Since `created_order` is just the json return of the order creation, we will need to use `get_order` from the orders API to get our order, and its download links.

In [ ]:
# You may need to wait a few minutes for the order to complete
order_id = created_order['id']
order_output = pl.orders.get_order(order_id)
print(f"Your order request state is: {order_output['state']}")

In [ ]:
# The link to download the order components is is the 'location' key of 'results'
order_output

## Alternative: Create the Order, monitor status until complete, then download the Order

Instead of manually checking our order's state with `get_order`, we can instead use an async function with a time tracking that will report when our order is ready. The order will be downloaded to the `output` directory

In [ ]:
# remember: "async def" to create the async coroutine
async def create_poll_and_download():
    async with Session(auth=auth) as sess:
        cl = OrdersClient(sess)

        # Use "reporting" to manage polling for order status
        with reporting.StateBar(state='creating') as bar:
            # create order via Orders client
            order = await cl.create_order(request)
            bar.update(state='created', order_id=order['id'])

            # poll...poll...poll...
            await cl.wait(order['id'], callback=bar.update_state)

        # if we get here that means the order completed. Yay! Download the files.
        cwd = os.getcwd()
        output_path = os.path.join(cwd, 'output')
        await cl.download_order(order['id'], output_path, progress_bar=True)

# remember: "await" to run the thing
await create_poll_and_download()